# Notebook 1: Introduction to Pandas and the Titanic Data Set
***

If you haven't at least skimmed the Numpy and Pandas tutorial, **STOP** and go do that now. 

In this notebook you'll apply some basic Pandas tools to explore the ubiquitous **Titanic** dataset. 

First, as always, we'll load Numpy and Pandas using their common aliases, np and pd. 

In [2]:
import numpy as np 
import pandas as pd

The data is stored in a .csv file (a format that lists data separated by commas) called clean_titanic_data.csv.  We'll import the data into Pandas using the read_csv( ) function.  Depending on how you're accessing this notebook you'll use one of two options to access the data.  If you've cloned the class GitHub repository then you'll have a local copy of the data set on your machine and you can use the local file path shown below.  If you are using Azure Notebooks, or have gotten this notebook in some other way, then you will use the the web-based file path shown below. 

In [ ]:
# Path to the data - select the path that works for you 
file_path = '../data/clean_titanic_data.csv'

# Load the data into a DataFrame 
df = pd.read_csv(file_path)

Take a look at the first few rows of the DataFrame using the head( ) method. 

In [ ]:
df.head()

From this you should see that each row in the DataFrame refers to a particular passenger on the Titanic.  The columns of the DataFrame give you specific information about each passenger.  The **PassengerId** is simply a unique identifier given to each passenger in the data set.  The rest of the attributes are more meaningful: 

- **Survived**: Indicates whether the passenger survived the sinking
- **Pclass**: Indicates the socio-economic status of the passenger (lower number means higher class)
- **Name**: The passenger's name 
- **Sex**: The passenger's sex 
- **Age**: The passenger's age
- **SibSp**: The number of siblings / spouses the passenger was traveling with 
- **Parch**: The number of children / parents the passenger was traveling with 
- **Ticket**: The passenger's ticket number 
- **Fare**: How much the passenger paid for their ticket 
- **Embarked**: The passenger's port of embarkation (C = Cherbourg, Q = Queenstown, S = Southampton)

### Exercise 1
***
Determine how many people survived the disaster

In [ ]:
people_survived = df["Survived"].sum()

print("{} people survived the disaster".format(people_survived))


### Exercise 2  
***
Determine how many men and how many women survived the disaster 

In [3]:
males = df.loc[df["Sex"]=='male']
male_survived = males["Survived"].sum()
females = df.loc[df["Sex"]=='female']
female_survived = females["Survived"].sum() 
print("{} men survived the disaster".format(male_survived))
print("{} women survived the disaster".format(female_survived))

NameError: name 'df' is not defined

### Exercise 3 
***
Determine how many children under the age of 12 survived the disaster. 

In [ ]:
children = df.loc[df["Age"]<12]
children_survived = children["Survived"].sum()
print("{} children survived the disaster".format(children_survived))

### Exercise 4 
***
The **SibSp** and **Parch** attributes tell us the number of siblings/spouses and parents/children each passenger had on board.  Create a new column in the DataFrame called **Family** that indicates how many siblings/spouses/parents/children a passenger was traveling with. Then report how many people survived that were traveling with 3 or more family members. 

In [ ]:
family_survived = # TODO
print("{} people traveling with 3 or more family members survived the disaster".format(family_survived))

### Exercise 5 
***
In this exercise you will write some code to predict whether a person survived the disaster based on their information.  Obviously, you'll want to ignore the **Survived** attribute for this to avoid cheating. You'll store your predictions ($1$ if you predict survived, $0$ if you predict died) in a column of the DataFrame called **Prediction**.  You can then use the following function to see how accurate your prediction was. 

In [ ]:
def score_prediction(df):
    '''
    Function to score predictions.  
    Takes entire DataFrame as sole argument. 
    '''
    acc = (df["Survived"]==df["Prediction"]).sum() / len(df)
    print("Your accuracy is {0:.1f}%".format(100*acc))

There are several ways that we can accomplish this.  The first method we'll highlight is one that loops over every row in the DataFrame, makes a decision based on that row's attributes, and then sets the relevant prediction in the **Prediction** column.  As an example, we'll use a very naive heuristic that predicts that all males survive and all females die. 

In [ ]:
for passenger_index, passenger in df.iterrows():
    df.loc[passenger_index, "Prediction"] = 1 if df.loc[passenger_index, "Sex"] == 'male' else 0

We can check that our code actually did something using the head( ) function and observing that we do in fact have a column called **Prediction** populated with $1$'s and $0$'s. You can see that the $1$'s in the **Prediction** column do in-fact line up with "male" in the **Sex** column. 

In [ ]:
df.head(10)

Now, let's see how we did by passing our DataFrame into the score_prediction( ) function. 

In [ ]:
score_prediction(df)

And here we see that our naive prediction netted us a 22% prediction accuracy (which isn't very good, but you're going to make it better). 

OK, so looping over the data is one option, but in Python, unfortunately, it's not a very good option.  Python is an interpreted language, which means that loops are slow.  We didn't really notice it here, because our DataFrame only has around 700 rows in it, but on data sets with hundreds of thousands or millions of entries, loops can grind your day to a complete halt.  

It's better to use vectorized methods like Pandas apply( ) function combined with Python lambda functions.  One way to accomplish the same results as above is as follows: 

In [ ]:
df["Prediction"] = df["Sex"].apply(lambda s: 1 if s=="male" else 0)
df.head(10)

You can check that this produces the same result as the loop-based method. We can compare the speeds of the apply( ) method and the loop-based method using the Jupyter magic %timeit command.  


In [ ]:
# Loop-based method 
print("Timing loop-based method: ")
%timeit for passenger_index, passenger in df.iterrows(): df.loc[passenger_index, "Prediction"] = 1 if df.loc[passenger_index, "Sex"] == 'male' else 0
    
# Apply-based method 
print("Timing apply-based method: ")
%timeit df["Prediction"] = df["Sex"].apply(lambda s: 1 if s=="male" else 0)

Note that here ms is milliseconds and $\mu\textrm{s}$ is **micro**seconds.  You can hopefully see that the apply method is **tremendously** faster than the loop-based method. 

OK, let's do one more slightly more complicated example so that we can see how to use the apply( ) function with multiple inputs.  Suppose say that you want to predict that a person survived if they are male **AND** they were traveling alone (probably not a good heuristic but just go with it).  To do this we need values from both the **Sex** column and the **Family** column.  Here is the code

In [ ]:
df["Prediction"] = df.apply(lambda row: 1 if row["Sex"]=="male" and row["Family"]==0 else 0, axis=1)
df.head()

Here the apply( ) function is applied to the entire DataFrame and the object passed to the lambda function is an entire row of the DataFrame.  We can then carve off the elements from the columns we're interested in and do our thing.  Let's see how we did! 

In [ ]:
score_prediction(df)

**W00T!** Minor improvement! OK, so you're job is to explore the data and see if you can cook up a custom prediction heuristic that does better than 31.1%!  I've created a Titanic Leaderboard post on Piazza.  If you get an accuracy you're proud of, post it there along with a description of what you did! 